In [ ]:
from google.colab import files
uploaded = files.upload()


Saving spotify_millsongdata.csv to spotify_millsongdata.csv


In [ ]:
#librerias
import pandas as pd
import nltk
from nltk.stem.porter import PorterStemmer

In [ ]:
#leemos el archivo .csv y lo almacenamos en data
data = pd.read_csv("spotify_millsongdata.csv")

In [ ]:
#muestra los primeros 5 resultados
data.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [ ]:
#muestra los últimos 5 resultados
data.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [ ]:
#muestra las dimensiones de la tabla
data.shape

(57650, 4)

In [ ]:
#elimina la columna "link" y las canciones empiezan a contar desde 0
data = data.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [ ]:
data.head(10)

,artist,song,text
0,Kenny Loggins,This Is It,"There've been times in my life, \r\nI've been..."
1,XTC,Shake You Donkey Up,Here he come again \r\nDress in all that skin...
2,King Diamond,Nuns Have No Fun,"Upon a cross a nun will be hanged, she will be..."
3,Death,Overactive Imagination,"Your existence is a script, \r\nLife for you ..."
4,John Denver,Calypso,"To sail on a dream on a crystal clear ocean, ..."
5,Westlife,Fool Again,"Baby, I know the story, \r\nI've seen the pic..."
6,Phil Collins,Another Day In Paradise,She calls out to the man on the street \r\n'S...
7,Christmas Songs,(It Must've Been Ol') Santa Claus,"(Lyrics by Harry Connick, Jr.) \r\n \r\nI th..."
8,Nightwish,Gethsemane,Toll no bell for me father \r\nBut let this c...
9,Richard Marx,Beautiful Goodbye,Baby don't you look so sad \r\nYou know we ha...


In [ ]:
data['text'] = data['text'].str.lower().replace(r'^\w\s','').replace(r'\n','', regex =True)

In [ ]:
data.tail(5)

,artist,song,text
4995,Iggy Pop,Howl,howl \rhowl \rfuck it \ri feel like howling...
4996,Isley Brothers,Between The Sheets,hey girl ain't no mystery \rat least as far a...
4997,Free,All Right Now,there she stood in the street \rsmiling from ...
4998,Eagles,Please Come Home For Christmas,bells will be ringing this sad sad new years ...
4999,Ian Hunter,River Of Tears,"waiting on an elevator, in a hotel out in cali..."


In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer

In [ ]:
!pip install nltk
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
stemmer = PorterStemmer()

In [ ]:
def token(txt):
  token = nltk.word_tokenize(txt)
  a =[stemmer.stem(w) for w in token]
  return " ".join(a)

In [ ]:
token("you are beautiful, beauty")

'you are beauti , beauti'

In [ ]:
data['text'].apply(lambda x: token(x))

0       there 've been time in my life , i 've been wo...
1       here he come again dress in all that skin like...
2       upon a cross a nun will be hang , she will be ...
3       your exist is a script , life for you is a per...
4       to sail on a dream on a crystal clear ocean , ...
                              ...                        
4995    howl howl fuck it i feel like howl got nowher ...
4996    hey girl ai n't no mysteri at least as far as ...
4997    there she stood in the street smile from her h...
4998    bell will be ring thi sad sad new year oh what...
4999    wait on an elev , in a hotel out in california...
Name: text, Length: 5000, dtype: object

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
tfid = TfidfVectorizer(analyzer='word', stop_words='english')

In [ ]:
matrix =tfid.fit_transform(data['text'])

In [ ]:
similer = cosine_similarity(matrix)

In [ ]:
data[data['song']=='Waiting For The Man'].index[0]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
def recommender(song_name):
  idx = data[data['song']==song_name].index[0]
  distance = sorted(list(enumerate(similer[idx])), reverse=True, key = lambda x:x[1])
  song = []
  for s_id in distance[1:21]:
    song.append(data.iloc[s_id[0]].song)
  return song

In [ ]:
recommender("River Of Tears")

['River Of Love',
 'Brokedown Palace',
 'A River For Him',
 'After All These Years',
 'All Those Years Ago',
 'Water Of Love',
 'River',
 'The Hungry Years',
 'Revelation',
 'It Tears Me Up (Live)',
 'Roll On .',
 'Green River',
 'Running Bear',
 'Gravity',
 'Pick Up The Oars And Row',
 "God Gave Rock 'n' Roll To You",
 'Still Crazy After All These Years',
 'Nine Pound Hammer',
 'Folk Tale',
 'Some Heads Are Gonna Roll']

In [ ]:
import pickle

In [ ]:
pickle.dump(similer, open("similarity", "wb"))

In [ ]:
pickle.dump(data,open("data", "wb"))